# Overview
This notebook shows how to create a simple strategy from scratch. 

In [ ]:
%use roboquant
Welcome()

## Werewolf strategy

We'll start with creating a very silly but simple strategy that adheres to the following 3 rules: 

1. If it is a super moon => we buy stocks
2. If it is a micro moon => we sell our stocks so we have no more exposure
3. Otherwise => we don't do anything.

Btw, a supermoon occurs when the moon’s orbit is closest to Earth at the same time it is full. So visually it is a very big moon. 

The main purpose of this example is obviously not to create a useful strategy that will make you a huge profit. But rather to demonstrate how easy it is to develop your own stategy.


## Use external libraries
We can load any library from external repositories and import the required packages afterwards. If required we can even add repositories to the notebook. In the background the notebook kernel will download the required package and all it declared dependencies. 

In this case we download the *common-suncalc* package for calculating moon phases.

In [ ]:
@file:DependsOn("org.shredzone.commons:commons-suncalc:3.4")
import org.shredzone.commons.suncalc.*

## Coding our own Strategy

We're now ready to develop the custom strategy. In this case we subclass the convenience class *SingleAssetStrategy* that can be used for strategies that are only interested in a single asset at a time. We need to override the method with signature:

```kotlin
fun generate(priceAction: PriceAction, now: Instant): Signal?
```
    
We get as input the price action for that asset and a timestamp for when the event occured. We return a Signal that indicates if we want to buy or sell that asset. We return *null* if we don't want to do anything.

In [ ]:
class WerewolfStrategy(asset: Asset) : SingleAssetStrategy(asset) {

    override fun generate(priceAction: PriceAction, now: Instant): Signal? {
        val phase = MoonPhase.compute().on(now).execute()
        return when {
            phase.isSuperMoon -> Signal(asset, Rating.BUY)
            phase.isMicroMoon -> Signal(asset, Rating.SELL)
            else -> null
        }
    }
    
}

## Create a new roboquant and evaluate it
We are now ready to back-test our strategy. And we decided to test it against Apple stock. So what we need to do is:

- Use a data feed that contains historic prices for the Apple stock
- Create an instance of our strategy and tell it to use Apple stock
- Create an instance of a roboquant and evaluate it against the available data 

In [ ]:
val feed = CSVFeed("../data/US")
val apple = feed.find("AAPL")
val strategy = WerewolfStrategy(apple)
val roboquant =  Roboquant(strategy)

roboquant.run(feed)

## Get some insights
Let's start to see how the account looks like after the expirement has run. The account holds the latest state

In [ ]:
val account = roboquant.broker.account
account.summary()

By default we start a simulated broker account with a default of 1_000_000.00 USD cash. Lets see much realized profit we actually made with out trades:

In [ ]:
account.trades.realizedPnL()

## Lessons learned

* It is not that hard to develop a new strategy and back-test it. 
* Even the most silly strategies can **appear** to make a profit if testing is based on unsound assumptions. In this case running our back test solely against Apple stocks and only going long, we were almost sure to make a profit.  